Get fiducial w, cs2, etc.

In [1]:
import params as par
from background import *

from scipy import integrate
from scipy.interpolate import interp1d
import numpy as np
import matplotlib.pyplot as plt
import copy
import pickle
plt.rcParams.update(par.plot_params)

In [2]:
ℋ = lambda τ: 2*α*(α*τ/τr + 1)/(α**2 * (τ**2/τr) + 2*α*τ)

In [3]:
"""3-Fluid Mode Evolution"""

# First run-through to obtain time steps:
τ_solve = np.linspace(par.τ0, τ_rec, 20)
k_solve = np.linspace(par.k_low, par.k_high, par.K_SOLVE)

for i, k in enumerate(k_solve):
    # set initial conditions
    Φ0  = 1.0
    δɣ0 = -2*Φ0*(1 + 3*y(τ0)/16)
    vɣ0 = -k/(ℋ(τ0)) * (δɣ0/4 + (2*k**2 * (1 + y(τ0))*Φ0)/
                        (9*(ℋ(τ0))**2 * (4./3. + y(τ0))))
    δc0 = .75 * δɣ0
    vc0 = vɣ0
    δν0 = δɣ0
    vν0 = vɣ0
    
    # solve the ODE
    def DY(τ, Y):
        [Φ, δɣ, vɣ, δc, vc, δν, vν] = Y
        DΦ  = -ℋ(τ)*Φ + (3/2.*H0**2.*a(τ)**2/k)*\
        (4./3.*(Ωɣ(τ)*vɣ + Ων(τ)*vν) + Ωc(τ)*vc + Ωb(τ)*vɣ)
        
        Dδɣ = -4./3.*k*vɣ + 4*DΦ
        Dvɣ = (-ℋ(τ) * yb(τ)*vɣ + k*δɣ/3)/(
            4./3. + yb(τ)) + k*Φ
        
        Dδc = -k*vc + 3*DΦ
        Dvc = -ℋ(τ)*vc + k*Φ
        
        Dδν = -4./3.*k*vν + 4*DΦ
        Dvν = k*δν/4 + k*Φ
        
        return [DΦ, Dδɣ, Dvɣ, Dδc, Dvc, Dδν, Dvν]
    Y0 = [Φ0, δɣ0, vɣ0, δc0, vc0, δν0, vν0]
    out = integrate.solve_ivp(DY, [τ0, τ_rec], Y0,
                 method='RK45', max_step=τ_rec/par.NMIN)
    τ_run = out['t']
    if (len(τ_run) > len(τ_solve)):
        τ_solve = τ_run

In [15]:
print(τ_solve2[τ_solve2<τ0])
print(τ0)

[1.49253731e-08]
1.4925373134328357e-08


In [24]:
# Second run-through to actually obtain the mode evolution
# initialize arrays
N = len(τ_solve)
τ_solve2 = np.geomspace(τ0,τ_rec,N)

Φ = np.zeros((par.K_SOLVE, N))
δɣ = np.zeros((par.K_SOLVE, N))
vɣ = np.zeros((par.K_SOLVE, N))
δc = np.zeros((par.K_SOLVE, N))
vc = np.zeros((par.K_SOLVE, N))
δν = np.zeros((par.K_SOLVE, N))
vν = np.zeros((par.K_SOLVE, N))


for i, k in enumerate(k_solve):
    
    # set initial conditions
    Φ0  = 1.0
    δɣ0 = -2*Φ0*(1 + 3*y(τ0)/16)
    vɣ0 = -k/(ℋ(τ0)) * (δɣ0/4 + (2*k**2 * (1 + y(τ0))*Φ0)/
                        (9*(ℋ(τ0))**2 * (4./3. + y(τ0))))
    δc0 = .75 * δɣ0
    vc0 = vɣ0
    δν0 = δɣ0
    vν0 = vɣ0
    
    # solve the ODE
    def DY(τ, Y):
        [Φ, δɣ, vɣ, δc, vc, δν, vν] = Y
        DΦ  = -ℋ(τ)*Φ + (3/2.*H0**2.*a(τ)**2/k)*\
        (4./3.*(Ωɣ(τ)*vɣ + Ων(τ)*vν) + Ωc(τ)*vc + Ωb(τ)*vɣ)
        
        Dδɣ = -4./3.*k*vɣ + 4*DΦ
        Dvɣ = (-ℋ(τ) * yb(τ)*vɣ + k*δɣ/3)/(
            4./3. + yb(τ)) + k*Φ
        
        Dδc = -k*vc + 3*DΦ
        Dvc = -ℋ(τ)*vc + k*Φ
        
        Dδν = -4./3.*k*vν + 4*DΦ
        Dvν = k*δν/4 + k*Φ
        
        return [DΦ, Dδɣ, Dvɣ, Dδc, Dvc, Dδν, Dvν]
    Y0 = [Φ0, δɣ0, vɣ0, δc0, vc0, δν0, vν0]
    out = integrate.solve_ivp(DY, [τ_solve2[0], τ_rec], Y0,
        method='RK45',t_eval=τ_solve2, max_step=τ_rec/par.NMIN)
    sol = out['y']
    
    Φ[i,:]  = sol[0]
    δɣ[i,:] = sol[1]
    vɣ[i,:] = sol[2]
    δc[i,:] = sol[3]
    vc[i,:] = sol[4]
    δν[i,:] = sol[5]
    vν[i,:] = sol[6]

Φ_  = interp1d(τ_solve, Φ,fill_value="extrapolate")
δɣ_ = interp1d(τ_solve, δɣ,fill_value="extrapolate")
vɣ_ = interp1d(τ_solve, vɣ,fill_value="extrapolate")
δc_ = interp1d(τ_solve, δc,fill_value="extrapolate")
vc_ = interp1d(τ_solve, vc,fill_value="extrapolate")
δν_ = interp1d(τ_solve, δν,fill_value="extrapolate")
vν_ = interp1d(τ_solve, vν,fill_value="extrapolate")

In [5]:
"""Extract GDM functions"""

τ = τ_solve
wd = Ων(τ)/(3*Ωd(τ))
Dwd = np.gradient(wd,τ)
δd = (Ωc(τ)*δc + Ων(τ)*δν)/Ωd(τ)
vd = (Ωc(τ)*vc + 4/3*Ων(τ)*vν)/((1+wd)*Ωd(τ))
cs2d =(δν*Ων(τ)*1/3)/(δd*Ωd(τ)) 

δd_ = interp1d(τ,δd, fill_value="extrapolate")
vd_ = interp1d(τ,vd, fill_value="extrapolate")
wd_ = interp1d(τ,wd, fill_value="extrapolate")
Dwd_ = interp1d(τ,Dwd, fill_value="extrapolate")
cs2d_ = interp1d(τ,cs2d, fill_value="extrapolate")

In [6]:
"""Save Outputs"""

pickle.dump(τ_solve, open( "data/tau_solve.p", "wb" ) )
pickle.dump(k_solve, open( "data/k_solve.p", "wb" ) )

pickle.dump(wd, open( "data/w_d.p", "wb" ) )
pickle.dump(Dwd, open( "data/Dw_d.p", "wb" ) )
pickle.dump(cs2d, open( "data/cs2_d.p", "wb" ) )

pickle.dump(δd, open( "data/delta_d.p", "wb" ) )
pickle.dump(vd, open( "data/v_d.p", "wb" ) )
pickle.dump(δɣ, open( "data/delta_g.p", "wb" ) )
pickle.dump(vɣ, open( "data/v_g.p", "wb" ) )
pickle.dump(Φ, open( "data/phi.p", "wb" ) )

In [7]:
def τ_itgd(a):
    """conformal time integrand"""
    return  1/(a**2* H0 * np.sqrt(Ωm0*a**-3 + Ωr0*a**-4 + ΩΛ))

def get_τ(a):
    (τ,_)=integrate.quad(τ_itgd,0,a)
    return τ

τ_rec = get_τ(a_rec)
τ_eq = get_τ(a_eq)
τ_now = get_τ(1.0)

ℋ = lambda τ: 2*α*(α*τ/τr + 1)/(α**2 * (τ**2/τr))
a = lambda τ: a_eq*((α*τ/τr)**2 + 2*α*τ/τr)
y = lambda τ: a(τ)/a_eq
yb = lambda τ: 1.68*y(τ)*Ωb0/Ωm0

Ωb = lambda τ: Ωb0 * a(τ)**-3.
Ωc = lambda τ: Ωc0 * a(τ)**-3.
Ωɣ = lambda τ: Ωɣ0 * a(τ)**-4.
Ων = lambda τ: Ων0 * a(τ)**-4.
Ωm = lambda τ: Ωm0 * a(τ)**-3.
Ωr = lambda τ: Ωr0 * a(τ)**-4.
Ωd = lambda τ: Ωc(τ) + Ων(τ)

plt.plot(wd)
plt.plot(np.log(ℋ(τ)))

def Omega_d(τ):
plt.show()

IndentationError: expected an indented block (<ipython-input-7-2fdfcebf8740>, line 30)

In [ ]:
plt.plot